In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("path_to_datafile")

---

## Calculate Promoter Activity

Here we estimate promoter activity fllowing SI of [Locke 2011](https://doi.org/10.1126/science.1208144)

Calculates promoter activity by unit cell length in units of 1/min (set by units of dt and gamma)

In [13]:
def derivative(y, frm_interval=1, log = False):
    if y.size > 2:
        y = np.log(y) if log else y
        x = np.arange(y.size) * frm_interval
        p = np.polyfit(x, y, 1) #simply 1st order polynomial fit / regression (numpy.polyfit)
        ddt = p[0] #slope of first order fit in python p[0]
    else: 
        ddt = np.nan 
    return ddt 

In [14]:
frm_interval = 10./60 #frame interval in minutes
gamma = 0.0015 #protein degradation constant in 1/min

mu = df.groupby('lin_id_str')['Spinelength'].rolling(3, min_periods=2, center=True).apply(derivative, kwargs={'frm_interval':frm_interval, 'log':True}).reset_index(0,drop=True)
df['mu'] = mu

ddt_gfp = df.groupby('lin_id_str')['MeanIntensityGFP'].rolling(3, min_periods=2, center=True).apply(derivative, kwargs={'frm_interval':frm_interval}).reset_index(0,drop=True)
df['ddt_gfp'] = ddt_gfp

ddt_rfp = df.groupby('lin_id_str')['MeanIntensityRFP'].rolling(3, min_periods=2, center=True).apply(derivative, kwargs={'frm_interval':frm_interval}).reset_index(0,drop=True)
df['ddt_rfp'] = ddt_rfp


df['promoter_act_G'] = (gamma + df['mu'] ) * df['MeanIntensityGFP'] + df['ddt_gfp']
df['promoter_act_R'] = (gamma + df['mu'] ) * df['MeanIntensityRFP'] + df['ddt_rfp']



,Position,PositionIdx,Indices,Frame,Idx,Time,BacteriaLineage,NextDivisionFrame,PreviousDivisionFrame,SizeRatio,...,MeanIntensityRFP,MeanIntensityGFP,ChannelIdx,lin_id_str,Length,mu,ddt_gfp,ddt_rfp,promoter_act_G,promoter_act_R
0,xy00,0,0-0-0,0,0,0.0,A,4.0,NaN,NaN,...,209.4800,1053.1000,0,0-0-A,24.6917,NaN,NaN,NaN,NaN,NaN
1,xy00,0,0-0-1,0,1,0.0,B,4.0,NaN,NaN,...,189.7748,1135.8000,0,0-0-B,26.9725,NaN,NaN,NaN,NaN,NaN
2,xy00,0,0-0-2,0,2,0.0,C,1.0,NaN,NaN,...,200.7550,1156.4000,0,0-0-C,58.8917,NaN,NaN,NaN,NaN,NaN
3,xy00,0,0-0-3,0,3,0.0,D,NaN,NaN,NaN,...,198.7530,997.1145,0,0-0-D,30.2778,NaN,NaN,NaN,NaN,NaN
4,xy00,0,0-1-0,0,0,0.0,A,4.0,NaN,NaN,...,316.3908,1126.3000,1,0-1-A,39.5609,NaN,NaN,NaN,NaN,NaN


In [17]:
df.describe()

AttributeError: 'DataFrame' object has no attribute 'summarys'